In [1]:
import pandas as pd

df = pd.read_csv("train (1).csv")
print(df.head())


        text_id                                          full_text  grade  \
0  5AA45740A848  Dear, TEACHER_NAME\n\nI think phone policy at ...      8   
1  790891A35047  Dear, Principal\r\n\r\nIn my opinion, I think ...      8   
2  C254D74362A3  PHONES\n\nDear principal students should have ...      8   
3  0985602832CD  phones\n\ni think phones should be allowed in ...      8   
4  3AB3FC197550  Do you really think students need cell phones ...      8   

                  prompt  Overall  Cohesion  Syntax  Vocabulary  Phraseology  \
0  Cell phones at school      3.0       3.5     2.5         3.5          3.0   
1  Cell phones at school      3.0       2.5     3.0         4.0          3.5   
2  Cell phones at school      3.0       2.5     3.0         3.0          3.0   
3  Cell phones at school      3.0       2.0     2.5         3.0          3.0   
4  Cell phones at school      3.0       2.0     3.0         3.0          3.0   

   Grammar  Conventions  
0      3.5          2.5  
1   

In [2]:
from sklearn.preprocessing import MinMaxScaler

scores = df["Overall"].values.reshape(-1, 1)
scaler = MinMaxScaler()
scores_scaled = scaler.fit_transform(scores).flatten()


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df["full_text"].tolist(), scores_scaled, test_size=0.2, random_state=42
)


In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
from torch.utils.data import Dataset, DataLoader
import torch

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
import torch
from torch.utils.data import Dataset

class EssayDataset(Dataset):
    def __init__(self, texts, scores):
        self.encodings = tokenize(texts)
        self.scores = torch.tensor(scores, dtype=torch.float)

    def __len__(self):
        return len(self.scores)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.scores[idx]
        return item

train_dataset = EssayDataset(X_train, y_train)
test_dataset = EssayDataset(X_test, y_test)


In [6]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=1,
    problem_type="regression"
)
from torch.utils.data import DataLoader

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import mean_squared_error

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    rmse = mean_squared_error(labels, preds)
    return {"rmse": rmse}

training_args = TrainingArguments(
    output_dir="./bert_aes",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="rmse"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: You chose "Don't visualize my results"


Epoch,Training Loss,Validation Loss,Rmse
1,0.016800,0.014059,0.014059
2,0.010900,0.011391,0.011391


Epoch,Training Loss,Validation Loss,Rmse
1,0.016800,0.014059,0.014059
2,0.010900,0.011391,0.011391
3,0.007000,0.012019,0.012019


TrainOutput(global_step=3891, training_loss=0.013228190401715014, metrics={'train_runtime': 1985.4731, 'train_samples_per_second': 7.834, 'train_steps_per_second': 1.96, 'total_flos': 4092655719490560.0, 'train_loss': 0.013228190401715014, 'epoch': 3.0})

In [8]:
import transformers
print(transformers.__version__)


4.57.3


In [9]:
pred_output = trainer.predict(test_dataset)
pred_scores_scaled = pred_output.predictions.flatten()
pred_scores = scaler.inverse_transform(pred_scores_scaled.reshape(-1,1))
print("Predicted Scores (sample):", pred_scores[:10].flatten())



Predicted Scores (sample): [3.9037182 3.328626  4.2079153 3.3093932 3.7595956 2.619626  2.686494
 3.6890643 3.4587853 3.635363 ]
